In [1]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_file, output_server
import datetime as dt

In [2]:
data = pd.read_csv('../data/train.csv')
data.head()

/home/jdickerson/.local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [3]:
data['DateTime'] = pd.to_datetime(data['Date'])
data['Year'] = data['DateTime'].apply(lambda x: x.year)
data['Month'] = data['DateTime'].apply(lambda x: x.month)
data['Day'] = data['DateTime'].apply(lambda x: x.day)
data['WeekNumber'] = data['DateTime'].apply(lambda x: x.isocalendar()[1])

In [4]:
from sklearn.preprocessing import LabelEncoder
data.ix[:, ['StateHoliday']] = data.ix[:, ['StateHoliday']].astype(str)

data['StateHoliday'] = LabelEncoder().fit_transform(data['StateHoliday'])


In [5]:
to_predict = data.ix[data['DateTime'] > dt.datetime(2015, 5, 1), :]
old_data = data.ix[data['DateTime'] < dt.datetime(2015, 5, 1), :]

In [8]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(old_data.drop(['Sales', 'Date', 'DateTime'], axis=1), 
                                                    old_data['Sales'], 
                                                    test_size=0.25, 
                                                    random_state=8675309)

In [9]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

AB = AdaBoostRegressor(DecisionTreeRegressor(),
                       n_estimators = 100, 
                       random_state=8675309).fit(X_train, y_train)

In [10]:
to_predict.ix[:, 'predictedSales'] = AB.predict(to_predict.drop(['Sales', 'Date', 'DateTime'], axis=1))

/home/jdickerson/.local/lib/python2.7/site-packages/pandas/core/indexing.py:266: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/jdickerson/.local/lib/python2.7/site-packages/pandas/core/indexing.py:426: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [11]:
to_predict.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,DateTime,Year,Month,Day,WeekNumber,predictedSales
0,1,5,2015-07-31,5263,555,1,1,0,1,2015-07-31,2015,7,31,31,5966
1,2,5,2015-07-31,6064,625,1,1,0,1,2015-07-31,2015,7,31,31,5970
2,3,5,2015-07-31,8314,821,1,1,0,1,2015-07-31,2015,7,31,31,9257
3,4,5,2015-07-31,13995,1498,1,1,0,1,2015-07-31,2015,7,31,31,13150
4,5,5,2015-07-31,4822,559,1,1,0,1,2015-07-31,2015,7,31,31,5629


In [12]:
from sklearn.metrics import mean_squared_error
from math import sqrt

sqrt(mean_squared_error(to_predict['Sales'], to_predict['predictedSales']))

857.5899318461379

In [13]:
datetime_groups = old_data.ix[data['Open']==1, :].groupby('DateTime')

dates = []
sales = []
for x, y in datetime_groups:
    dates.append(x)
    sales.append(y['Sales'].mean())

In [14]:
pred_datetime_groups = to_predict.ix[data['Open']==1, :].groupby('DateTime')

pred_dates = []
pred_sales = []
actual_sales = []
for x, y in pred_datetime_groups:
    pred_dates.append(x)
    pred_sales.append(y['predictedSales'].mean())
    actual_sales.append(y['Sales'].mean())

In [17]:
output_server("Projected Sales vs. Actual")

p = figure(x_axis_type = "datetime", 
           y_axis_type="log",  
           plot_width=1000, 
           plot_height=700, 
           title="Projected Sales vs. Actual")
p.xaxis.axis_label="Date"
p.yaxis.axis_label="Sales (log scale)"

p.line(dates, sales , line_width=2, color='#1F78B4')
p.line(pred_dates, pred_sales, color='#FB9A99')
p.line(pred_dates[:50], actual_sales[:50], color='#01DF01')


show(p)



Using saved session configuration for http://localhost:5006/
To override, pass 'load_from_config=False' to Session


In [68]:
graph_data = {"Algorithm": ["Decision Tree", "Neural Net", "Linear SVR", "Adaboosted Tree", "Gradient Boost"],
              "RMSE": [1115.85, 3865.83, 1540.97, 789.80, 1540.9]
             }



# graph_data = {"x": [1115.85, 3865.83, 1540.97, 789.80, 1540.9]}
graph_cats = ["Decision Tree", "Neural Net", "Linear SVR", "Adaboosted Tree", "Gradient Boost"]

df = pd.DataFrame(graph_data)
df


,Algorithm,RMSE
0,Decision Tree,1115.85
1,Neural Net,3865.83
2,Linear SVR,1540.97
3,Adaboosted Tree,789.80
4,Gradient Boost,1540.90


In [69]:
from bokeh.charts import Bar, output_file

b = Bar(df, label='Algorithm', values="RMSE", title='RMSE of Algorithms', xlabel='Algorithm', ylabel='RMSE')

output_file("bar.html")

show(b)

ERROR:/usr/local/lib/python2.7/dist-packages/bokeh/validation/check.pyc:W-1001 (NO_GLYPH_RENDERERS): Plot has no glyph renderers: Figure, ViewModel:Plot, ref _id: c6f2476b-ffc4-4f6f-bb65-34eca3368553


In [65]:
full_dates_grouped = data.groupby(['Promo'])

for x, y in full_dates_grouped:
    if x == 0:
        promo_0_sales = y['Sales'].mean()
    else:
        promo_1_sales = y['Sales'].mean()

In [67]:
promo_df = {"Promo": [0, 1], "Sales": [promo_0_sales, promo_1_sales]}

In [77]:
from bokeh.charts import Bar, output_file
output_server("Promo Effects")

p = Bar(promo_df, label="Promo", values="Sales", function="mean")

output_file("promo.html")

show(p)


Using saved session configuration for http://localhost:5006/
To override, pass 'load_from_config=False' to Session


AttributeError: unexpected attribute 'function' to ChartOptions, possible attributes are filename, height, id, legend, name, notebook, responsive, server, session, tags, title, title_text_font_size, tools, width, xgrid, xlabel, xscale, ygrid, ylabel or yscale

In [50]:
[x for x in promo_1_sales if np.isnan(x) == False]

[87.284560143626564,
 6233.0304932735426,
 5693.1121076233185,
 5954.2188340807179,
 5337.7515695067268,
 129.06188340807174,
 5396.8529147982063,
 129.19461883408073,
 5279.6304932735429,
 4944.0278026905826,
 4747.1031390134531,
 4830.9991031390136,
 5504.044843049327,
 5390.3067264573992,
 135.96412556053812,
 5245.5336322869953,
 128.11479820627804,
 5272.3784753363225,
 4890.2852017937221,
 5300.2502242152468,
 6339.5219730941708,
 6460.0959641255604,
 6082.2789237668158,
 153.86995515695068,
 5525.5775784753359,
 154.76591928251122,
 4948.0358744394616,
 4989.6887892376681,
 5538.2484304932732,
 5759.9031390134533,
 5609.913004484305,
 5526.5847533632286,
 156.47264573991032,
 5056.4529147982066,
 142.65201793721974,
 5036.2017937219734,
 5086.3121076233183,
 5388.7919282511211,
 6375.7596412556049,
 6497.0681614349778,
 6080.134529147982,
 175.09327354260088,
 5436.2179372197306,
 147.90762331838565,
 5555.6394618834083,
 4787.8188340807174,
 5229.3605381165917,
 5469.1417040358